# Exercise 9

## Construction Phase

In [ ]:
import tensorflow as tf
import numpy as np

n_inputs  = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

Connect the network.

In [ ]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope('dnn'):
    hidden1 = fully_connected(X,       n_hidden1, scope='hidden1')
    hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')
    logits  = fully_connected(hidden2, n_outputs, scope='outputs', activation_fn=None)

Define the cost function (loss).

In [ ]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss     = tf.reduce_mean(xentropy, name='loss')

Define a Gradient Descent Optimizer to minimize the cost function.

In [ ]:
learning_rate = 0.01

with tf.name_scope('train'):
    optimizer   = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

Use accuracy as a measure for evaluationg the model's performance.

In [ ]:
with tf.name_scope('eval'):
    correct  = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()

## Execution Phase

Use TensorFlow to load the MNIST data set.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

Prepare logging.

In [ ]:
from datetime import datetime

now         = datetime.utcnow().strftime('%y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir      = '{}/run-{}'.format(root_logdir, now)

In [ ]:
loss_summary = tf.summary.scalar('loss', loss)
acc_summary  = tf.summary.scalar('accuracy', accuracy)
file_writer  = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
n_epochs   = 400
batch_size = 50

In [ ]:
n_batches = mnist.train.num_examples // batch_size

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        print("\repoch {}/{}".format(epoch + 1, n_epochs), end='')
        for iteration in range(n_batches):
            
            # Train the network on a batch.
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            # Every once in a while send a summary to TensorBoard.
            if iteration % 10 == 0:
                loss_summ, acc_summ = sess.run(
                    [loss_summary, acc_summary],
                    feed_dict={X: mnist.test.images, y: mnist.test.labels})

                step = epoch * n_batches + iteration
                file_writer.add_summary(loss_summ, step)
                file_writer.add_summary(acc_summ, step)

file_writer.close()